In [1]:
import time
import requests
import urllib
import json
from datetime import datetime
import dateutil.parser
from splinter import Browser
from tosConfig import consumerKey, passWord, userName
from tdAuthen import accesstoken

In [2]:
authen_data = accesstoken(consumerKey = consumerKey, passWord = passWord, userName = userName)

Pulled Code, grabbing access token....


In [3]:
import datetime
def unix_time_millis(dt):
    epoch = datetime.datetime.utcfromtimestamp(0)
    return (dt - epoch).total_seconds() * 1000.0

In [4]:
# define endpoint which is the User Principles emdpoint

endpoint = 'https://api.tdameritrade.com/v1/userprincipals'
headers = {'Authorization':'Bearer {}'.format(authen_data['access_token'])}

# define the paramenters for the endpoint
params = {'fields': 'streamerSubscriptionKeys,streamerConnectionInfo'}

# make a requests
content = requests.get(url = endpoint, params = params, headers = headers)
userPrincipalsResponse = content.json()

# defining items needed to make requests to login
tokenTimeStamp = userPrincipalsResponse['streamerInfo']['tokenTimestamp']
date = dateutil.parser.parse(tokenTimeStamp, ignoretz = True)
tokenTimeStampAsMs = unix_time_millis(date)

credentials = {'userid':userPrincipalsResponse['accounts'][0]['accountId'],
              'token':userPrincipalsResponse['streamerInfo']['token'],
              'company':userPrincipalsResponse['accounts'][0]['company'],
              'segment':userPrincipalsResponse['accounts'][0]['segment'],
              'cddomain':userPrincipalsResponse['accounts'][0]['accountCdDomainId'],
              'usergroup':userPrincipalsResponse['streamerInfo']['userGroup'],
                'accesslevel': userPrincipalsResponse['streamerInfo']['accessLevel'],
              'authorized':'Y',
              'timestamp':int(tokenTimeStampAsMs),
              'appid':userPrincipalsResponse['streamerInfo']['appId'],
              'acl':userPrincipalsResponse['streamerInfo']['acl']}



In [38]:
# defining login request
login_request = {"requests":[{"service": "ADMIN",
                              "requestid": "0",
                              "command": "LOGIN",
                              "account":userPrincipalsResponse['accounts'][0]['accountId'],
                              "source":userPrincipalsResponse['streamerInfo']['appId'],
                              "parameters": {"credential": urllib.parse.urlencode(credentials),
                                             "token":userPrincipalsResponse['streamerInfo']['token'],
                                             "version": "1.0"}}]}

data_request = {"request": [{"service":"TIMESALE_EQUITY",
                                 "requestid": "1",
                                "command": "SUBS",
                                "account":userPrincipalsResponse['accounts'][0]['accountId'],
                                "source": userPrincipalsResponse['streamerInfo']['appId'],
                                "parameters": {
                                "keys": "AAPL",
                                "fields": "0, 1,2,3,4"}}]} 

login_encoded = json.dumps(login_request)
data_encoded = json.dumps(data_request)

'{"requests": [{"service": "ADMIN", "requestid": "0", "command": "LOGIN", "account": "496550006", "source": "lopezproceso", "parameters": {"credential": "userid=496550006&token=5427eef86c90c51b74252a4507c1f43748275a61&company=AMER&segment=ADVNCED&cddomain=A000000077544124&usergroup=ACCT&accesslevel=ACCT&authorized=Y&timestamp=1632441166000&appid=lopezproceso&acl=AKBPCNCVDRDTDWESF7G1G3G5G7GKGLH1H3H5LTM1MAMROLPNQSRFSDTETFTOTTUAURXBXNXOD2D4D6D8E2E4E6E8F2F4F6H7I1", "token": "5427eef86c90c51b74252a4507c1f43748275a61", "version": "1.0"}}]}'

In [50]:
import websockets
from websockets import client
import asyncio
# This connects to the database
class WebSocketClient():
    
    def __init__(self):
        self.data_holder = []
        self.file = open('td_data.txt', 'a')
    
# #     def database_connection(self):
# #         server = 'SERVER_NAME'
# #         databse = 'DATABASE_NAME'
# #         sql_driver = '{ODBC Driver 17 for SQL Servers}'
# # #     def database_connect(self)
# # #     server = 'blah'
# # #     database = ''

    
    def csv_insert(self, data_tuple):
        data = date_tuple
        file = open('td_data.txt','w')
        file.write(data_tuple)
        file.close()
    
    async def connect(self):
        
        uri = "wss://" + userPrincipalsResponse['streamerInfo']['streamerSocketUrl']
        self.connection = await websockets.client.connect(uri)
        if self.connection.open:
            print("Connection established. Client correctly connected")
            return self.connection
    
#     async def sendMessage(self, message):
#         await self.connection.send(message)
        
    
#     async def receiveMessage(self, connection):
        
#         while True: 
            
#             try:
#                 #grab and decode the message
#                 message = await connection.recv()
#                 message_decoded = json.loads(message)
                
#                 if 'data' in message_decoded.keys():
                    
#                     # grab the data
#                     data = message_decoded['data'][0]
#                     data_tuple = (data['service'], str(data['timestamp']), data['command'])
                    
#                     # insert the data
#                     self.csv_insert(data_tuple)
                
#                 print('-'*20)
#                 print('Recieved message from server: ' + str(message))
                
                
#             except websockets.exceptions.ConnectionClosed:
#                 print('Connection with server closed')
#                 break
    
#     async def heartbeat(self, connection):
#         while True:
#             try: 
#                 await connection.send('ping')
#                 await asyncio.sleep(5)
#             except websockets.exceptions.ConnectionClosed:
#                 print('Connection with server closed')
#                 break


In [51]:
import nest_asyncio
nest_asyncio.apply()


if __name__ == '__main__':
    
    # Creating client object
    client = WebSocketClient()
    
    loop = asyncio.get_event_loop()
    
    # Start connection and get client connection protocol
    connection = loop.run_until_complete(client.connect())
    
    # Start listener and heartbeat 
    tasks = [asyncio.ensure_future(client.receiveMessage(connection)),
             asyncio.ensure_future(client.sendMessage(login_encoded)),
             asyncio.ensure_future(client.receiveMessage(connection)),
             asyncio.ensure_future(client.sendMessage(data_encoded)),
             asyncio.ensure_future(client.receiveMessage(connection))]

    loop.run_until_complete(asyncio.wait(tasks))

InvalidStatusCode: server rejected WebSocket connection: HTTP 404